In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import OrderedDict
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TargetScanHuman = '/content/drive/MyDrive/miRLY/TargetScanHuman8.0.txt'
TarBase = '/content/drive/MyDrive/miRLY/TarBase v7.0.csv'
miRecords = '/content/drive/MyDrive/miRLY/miRecords_version4.xls'
miRDB = '/content/drive/MyDrive/miRLY/miRDB_v6.0_prediction_result.txt'
miRtar = '/content/drive/MyDrive/miRLY/miRtar.txt'

TSH = pd.read_table(TargetScanHuman)
TB = pd.read_table(TarBase)
MRC = pd.read_excel(miRecords)
MRB = pd.read_table(miRDB, error_bad_lines = False, engine="python")
MIT = pd.read_table(miRtar, error_bad_lines = False, engine="python")

Skipping line 402: '	' expected after '"'


In [ ]:
TSH.columns = ['Ensembl_Gene_ID', 'Gene_Name', 'Transcript_ID', 'Gene_Tax_ID', 'miRNA_ID',
       'Site_Type', 'UTR_start', 'UTR_end', 'context++_score',
       'context++_score_percentile', 'weighted_context++_score',
       'weighted_context++ _score_percentile', 'Predicted_relative_KD']
TSH = TSH.loc[:,['Ensembl_Gene_ID', 'Gene_Name', 'Gene_Tax_ID', 'miRNA_ID']]
condition = (TSH.Gene_Tax_ID == 9606)
TSH = TSH.loc[condition, ['Ensembl_Gene_ID', 'Gene_Name', 'miRNA_ID']]

TSH['Gene_Name'] = TSH['Gene_Name'].str.upper()
TSH['Gene_Name'] = TSH['Gene_Name'].str.strip()
TSH['Gene_Name'] = TSH['Gene_Name'].str.replace('_',"-")
TSH['Gene_Name'] = TSH['Gene_Name'].str.replace(' ',"")

In [ ]:
TB.columns = ['geneId', 'Gene_Name', 'miRNA_ID', 'species', 'cell_line', 'tissue', 'category', 'method', 'positive_negative', 'direct_indirect', 'up_down', 'state']
TB = TB.loc[:,['Gene_Name', 'miRNA_ID', 'species']]
condition = (TB.species == "Homo sapiens")
TB = TB.loc[condition, ['Gene_Name', 'miRNA_ID']]
TB = TB[~TB['Gene_Name'].str.contains('ENS', na=False, case=False)]
TB = TB[~TB['Gene_Name'].str.contains('XM_', na=False, case=False)]
TB = TB[~TB['Gene_Name'].str.contains('NM_', na=False, case=False)]

TB['Gene_Name'] = TB['Gene_Name'].str.upper()
TB['Gene_Name'] = TB['Gene_Name'].str.strip()
TB['Gene_Name'] = TB['Gene_Name'].str.replace('_',"-")
TB['Gene_Name'] = TB['Gene_Name'].str.replace(' ',"")

In [ ]:
MRC.columns = ['Pubmed_ID', 'Target gene_species_scientific', 'Gene_Name',
       'RefSeq_Acc_ID', 'Target site_number', 'miRNA_species',
       'miRNA_ID', 'miRNA_regulation', 'Reporter_target gene/region',
       'Reporter link element', 'Test_method_inter', 'Target gene mRNA_level',
       'Original description', 'Mutation_target region',
       'Post mutation_method', 'Original description_mutation_region',
       'Target site_position', 'miRNA_regulation_site', 'Reporter_target site',
       'Reporter link element.1', 'Test_method_inter_site',
       'Original description_inter_site', 'Mutation_target site',
       'Post mutation_method_site', 'Original description_mutation_site', 'Additional note']
MRC = MRC.loc[:,['Pubmed_ID', 'Gene_Name', 'RefSeq_Acc_ID', 'miRNA_species', 'miRNA_ID']]
condition = (MRC.miRNA_species == "Homo sapiens")
MRC = MRC.loc[condition, ['Pubmed_ID', 'Gene_Name', 'RefSeq_Acc_ID', 'miRNA_ID']]

MRC = MRC[~MRC['Gene_Name'].str.contains('2012.9.2', na=False, case=False)]
MRC['Gene_Name'] = MRC['Gene_Name'].str.upper()
MRC['Gene_Name'] = MRC['Gene_Name'].str.strip()
MRC['Gene_Name'] = MRC['Gene_Name'].str.replace('_',"-")
MRC['Gene_Name'] = MRC['Gene_Name'].str.replace(' ',"")

In [ ]:
MRB.columns=['miRNA_ID','RefSeq_Acc_ID','Score']
MRB.loc[6831594] = {'miRNA_ID' : 'cfa-miR-1185', 'RefSeq_Acc_ID' : 'XM_537211', 'Score' : '59.3438099752'}
MRB = MRB.loc[:,['miRNA_ID', 'RefSeq_Acc_ID']]
MRB = MRB[MRB['miRNA_ID'].str.contains('hsa')]

In [ ]:
df1 = TSH.loc[:,['miRNA_ID', 'Gene_Name']]
df2 = TB.loc[:,['miRNA_ID', 'Gene_Name']]
df3 = MRC.loc[:,['miRNA_ID', 'Gene_Name']]
df4 = MRB.loc[:,['miRNA_ID']]

mirna_id = pd.concat([df1, df2, df3])
mirna_id = mirna_id.drop_duplicates()
mirna_id = mirna_id.dropna(axis = 0)
mirna_id = mirna_id.drop_duplicates()
mirna_id.to_csv('/content/drive/MyDrive/miRLY/mirna_id.csv')

mirnaid = mirna_id.loc[:,['miRNA_ID']]
mirnaid = mirnaid.drop_duplicates()
mirnaid.to_csv('/content/drive/MyDrive/miRLY/miRNA_ID.csv')

Target = mirna_id.loc[:,['Gene_Name']]
Target = Target.drop_duplicates()
Target.to_csv('/content/drive/MyDrive/miRLY/target.csv')

In [ ]:
GC = '/content/drive/MyDrive/miRLY/geneID_conversion_fin_unique_ensemb.tsv'
gc = pd.read_table(GC)
gc.columns = ['Unnamed: 0', 'Gene_Name', 'Gene_ID', 'Ensembl_ID', 'OMIM']
gc = gc.loc[:,['Gene_Name', 'Gene_ID', 'Ensembl_ID', 'OMIM']]
gc = gc[~gc['Gene_Name'].str.contains('ENS', na=False, case=False)]
gc = gc[~gc['Gene_Name'].str.contains('XM-', na=False, case=False)]
gc = gc[~gc['Gene_Name'].str.contains('NM-', na=False, case=False)]
gc['Gene_Name'] = gc['Gene_Name'].str.replace(' ',"")

DATA = pd.merge(mirna_id, gc, on = 'Gene_Name', how = "outer")
data = DATA.loc[:,['miRNA_ID', 'Gene_Name']]
data = data.dropna(axis = 0)

DATAF = pd.merge(data, gc, on = 'Gene_Name', how = "inner")
DATAF = DATAF.drop_duplicates()
DATAF.to_csv('/content/drive/MyDrive/miRLY/DATA.csv')

In [ ]:
#print(DATAF)

              miRNA_ID Gene_Name      Gene_ID       Ensembl_ID      OMIM
0          hsa-miR-23c      A1BG          1.0  ENSG00000121410  138670.0
1       hsa-miR-23b-3p      A1BG          1.0  ENSG00000121410  138670.0
2       hsa-miR-23a-3p      A1BG          1.0  ENSG00000121410  138670.0
3         hsa-miR-7-5p      A1CF      29974.0  ENSG00000148584  618199.0
4       hsa-miR-30b-5p      A1CF      29974.0  ENSG00000148584  618199.0
...                ...       ...          ...              ...       ...
473644     hsa-miR-424      VEGF  100329011.0              NaN       NaN
473645     hsa-miR-758     MYTL1          NaN              NaN       NaN
473646      hsa-miR-21      NT-3    5650150.0              NaN       NaN
473647     hsa-miR-622     K-RAS       3845.0  ENSG00000133703  190070.0
473648    hsa-miR-125b    4E-BP1       1978.0  ENSG00000187840  602223.0

[473649 rows x 5 columns]


In [ ]:
#gc.isnull().sum()

Gene_Name      515
Gene_ID        930
Ensembl_ID    1120
OMIM          4767
dtype: int64

In [ ]:
#DATAF.notnull().sum()

miRNA_ID      473649
Gene_Name     473649
Gene_ID       472564
Ensembl_ID    466161
OMIM          415156
dtype: int64

In [ ]:
#DATAF['miRNA_ID'].value_counts()

hsa-miR-34a-5p     4532
hsa-miR-16-5p      3684
hsa-miR-26b-5p     3290
hsa-miR-27a-3p     3214
hsa-miR-335-5p     3165
                   ... 
hsa-miR-3177-3p       1
hsa-miR-520g          1
[miR-143]             1
hsa-miR-371a-3p       1
hsa-mir-491           1
Name: miRNA_ID, Length: 1316, dtype: int64